<font size=50 color=darkblue>Min Cut MILP</font>

# Problem modelling with DOcplex

## Import necessary module

- DOcplex will be used to model and solve the Min-Cut MILP

In [ ]:
from docplex.mp.model import Model

## Min-Cut MILP <font size=3>(variables are colored blue)</font>
**Minimize**
### $$\sum_{(u,v)\in \mathcal{E}}\mu_{(u,v)}\cdot \color{blue}x_{(u,v)}$$
**Subject to**
### \begin{align*}
{\color{blue}z}_{\color{red}(t)} - {\color{blue}z}_{\color{green}(s)}&=1&\\
{\color{blue}x_{(u,v)}} &\ge {\color{blue}z_{(v)}} - {\color{blue}z_{(u)}},&\forall(u,v)\in \mathcal{E}\\
{\color{blue}x_{(u,v)}}&\in \left\{0,1\right\},&\forall(u,v)\in \mathcal{E}\\
{\color{blue}z_{(v)}} &\in \mathbb{R},&\forall v\in \mathcal{V}\\
\end{align*}

## Display the network
<img src='img/mc.png' width=700/>

## Define the model input
### \begin{align*}
\mathcal{V} =\,& \{0, 1, 2, 3, 4, 5, 6, 7\}&\\
\mathcal{E} =\,& \{(0,1), (0,2), (0,3), (1,2), (1,4), (1,5), (2,3), (2,5), (3,6), (4,5), (4,7), (5,6), (5,7), (6,2), (6,7)\}&\\
{\color{green}s} =\,& 0&\\
{\color{red}t} =\,& 7&
\end{align*}

In [ ]:
V = [0, 1, 2, 3, 4, 5, 6, 7]
E = [(0,1), (0,2), (0,3), (1,2), (1,4), (1,5), (2,3), (2,5), (3,6), (4,5), (4,7), (5,6), (5,7), (6,2), (6,7)]
s, t = 0, 7

## Define link capacities $\mu_{(e)},\,\forall e\in \mathcal{E}$, subscriptable by link indices

In [ ]:
mu = {e: cap for e,cap in zip(E, [10, 5, 15, 4, 9, 15, 4, 8, 30, 15, 10, 15, 10, 6, 10])}

## Create the MILP model for Min-Cut problem using DOcplex

In [ ]:
mc_MILP = Model(name='Minimum Cut')

## Define decision variables
## \begin{align*}
{\color{blue}x_{(u,v)}}&\in \left\{0,1\right\},&\forall(u,v)\in \mathcal{E}\\
{\color{blue}z_{(v)}} &\in \mathbb{R},&\forall v\in \mathcal{V}\\
\end{align*}

In [ ]:
mc_MILP.clear() # This line is optional. Its purpose is to avoid variable duplicates
x = {(u,v): mc_MILP.binary_var(name=f'x({u},{v})') for u,v in E}
z = {v: mc_MILP.continuous_var(name=f'z({v})') for v in V}

## Define node cut constraints, subscriptable by link indices
## \begin{align*}
{\color{blue}x_{(u,v)}} &\ge {\color{blue}z_{(v)}} - {\color{blue}z_{(u)}},&\forall(u,v)\in \mathcal{E}\\
{\color{blue}z}_{\color{red}(t)} - {\color{blue}z}_{\color{green}(s)}&=1&\\
\end{align*}

In [ ]:
ct = {(u,v): mc_MILP.add_constraint(ct=x[u,v] >= z[v] - z[u]) for (u,v) in E}
ct[t,s] = mc_MILP.add_constraint(ct=z[t] - z[s] == 1)

## Define objective function $$\textbf{minimize}\qquad \sum_{(u,v)\in \mathcal{E}}\mu_{(u,v)}\cdot \color{blue}x_{(u,v)}$$

In [ ]:
mc_MILP.minimize(mc_MILP.sum(mu[e]*x[e] for e in E))

## Summarize the model

In [ ]:
mc_MILP.print_information()

## Solve the MILP and display the result

In [ ]:
mc_sol = mc_MILP.solve()
if mc_sol:
    mc_sol.display()

# Result Visualization

## Import visualization modules
- `igraph`, `numpy`, `matplotlib`, `scipy`

In [ ]:
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.spatial import ConvexHull

## For convenience, extract the solution to dictionaries named `sol_x` and `sol_z`

In [ ]:
sol_x = mc_sol.get_value_dict(x)
sol_z = mc_sol.get_value_dict(z)
print(f'{sol_x = }')
print(f'{sol_z = }')

## Instantiate a `Graph` object with module `igraph`
### Notes
- __*Node(s)*__ is/are called __*vertex/vertices*__ in `igraph`
- __*Link(s)*__ is/are called __*edge/edges*__ in `igraph`
- The edge list is sufficient to instantiate a `Graph` object. The vertex list is automatically inferred by `igraph` (based on the tails/heads' indices).

In [ ]:
g = ig.Graph(edges=E, directed=True)

## Visualize the graph

In [ ]:
g.vs['label'] = g.vs.indices
g.vs['size'] = 50
g.vs['color'] = 'white'
g.vs[s,t]['color'] = 'green', 'red'

g.es['label'] = [mu[e] for e in E]
g.es['arrow_size'] = [10 if sol_x[e] > 0 else 6 for e in E]
g.es['arrow_width'] = [10 if sol_x[e] > 0 else None for e in E]
g.es['width'] = [3 if sol_x[e] > 0 else 0.5 for e in E]
g.es['color'] = ['darkblue' if sol_x[e] > 0 else None for e in E]
g.es['label_size'] = 15

fig, ax = plt.subplots()
fig.set_size_inches(10,10)

layout = g.layout('fr')
coords = np.asarray(layout.coords)
p = ig.plot(g, layout=layout, edge_label=g.es['label'], edge_background='white', target=ax)

bboxes = [path.get_extents().transformed(ax.transData.inverted()) for path in p.get_vertices().get_paths()]
radii = [max(bbox.height, bbox.width)/2 for bbox in bboxes]

S_circle_pads = [plt.Circle(coords[v], radii[v]*1.5) for v in V if sol_z[v] == sol_z[s]]
S_pts = np.asarray([pt for c in S_circle_pads for pt in c.get_patch_transform().transform(c.get_path().vertices)])

centroid = np.atleast_2d(S_pts[ConvexHull(S_pts).vertices].mean(0))

cross_pts = [(2/3)*coords[u, :] + (1/3)*coords[v, :] for u, v in E if sol_x[u, v] == 1]

hull = ConvexHull(bndry_pts:=np.vstack((S_pts, cross_pts)))
bndry_pts = bndry_pts[hull.vertices]
bndry_pts = np.unique(np.vstack((bndry_pts, cross_pts)), axis=0)
bndry_pts = bndry_pts[np.argsort(np.arctan2(*(bndry_pts - centroid).T))]

tck, u = interpolate.splprep(np.tile(bndry_pts, (20,1)).T, s=0)
out = interpolate.splev(np.linspace(0, 1, 1000), tck)

plt.fill(out[0], out[1], c='green', alpha=.3, ec=None)

plt.show()